In [4]:
import cv2
import os
import sys

# Specify the correct path to the video file
video_path = 'C:/Users/user/anaconda3/GoPro.mp4'
video = cv2.VideoCapture(video_path)

if not video.isOpened():
    print("Error: Could not open video file.")
    sys.exit()

# Folder for saving results
output_folder_kcf = 'C:/Users/user/anaconda3/output_kcf'
output_folder_csrt = 'C:/Users/user/anaconda3/output_csrt'

if not os.path.exists(output_folder_kcf):
    os.makedirs(output_folder_kcf)

if not os.path.exists(output_folder_csrt):
    os.makedirs(output_folder_csrt)

# Read the first frame
ret, frame = video.read()
if not ret:
    print("Failed to read video")
    sys.exit()

# Select the initial bounding box
bbox = cv2.selectROI("Frame", frame, False)
cv2.destroyAllWindows()  # Close the ROI selection window

# Define the function to create trackers
def create_tracker(tracker_type):
    if tracker_type == 'BOOSTING':
        return cv2.legacy.TrackerBoosting_create()
    elif tracker_type == 'MIL':
        return cv2.legacy.TrackerMIL_create()
    elif tracker_type == 'KCF':
        return cv2.legacy.TrackerKCF_create()
    elif tracker_type == 'TLD':
        return cv2.legacy.TrackerTLD_create()
    elif tracker_type == 'MEDIANFLOW':
        return cv2.legacy.TrackerMedianFlow_create()
    elif tracker_type == 'GOTURN':
        return cv2.legacy.TrackerGOTURN_create()
    elif tracker_type == 'MOSSE':
        return cv2.legacy.TrackerMOSSE_create()
    elif tracker_type == "CSRT":
        return cv2.legacy.TrackerCSRT_create()
    else:
        raise ValueError(f"Unknown tracker type: {tracker_type}")

# Initialize trackers
tracker_kcf = create_tracker('KCF')
tracker_csrt = create_tracker('CSRT')

# Initialize trackers with the first frame and bounding box
tracker_kcf.init(frame, bbox)
tracker_csrt.init(frame, bbox)

# Function to run and save the tracker
def run_tracker(tracker, tracker_name, output_folder):
    video.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset video to the beginning
    for i in range(15):
        ret, frame = video.read()
        if not ret:
            break

        # Update the tracker
        ok, bbox = tracker.update(frame)
        if ok:
            # Draw the bounding box
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (0, 255, 0), 2, 1)
        
        # Save the image
        output_path = os.path.join(output_folder, f"frame_{i:04d}.jpg")
        cv2.imwrite(output_path, frame)

# Run the KCF tracker and save the results
run_tracker(tracker_kcf, 'KCF', output_folder_kcf)

# Run the CSRT tracker and save the results
run_tracker(tracker_csrt, 'CSRT', output_folder_csrt)